In [5]:
try:
    import unicodecsv
except:
    !pip install unicodecsv
    import unicodecsv

In [6]:
try:
    import numpy
except:
    !pip install numpy
    import numpy

In [7]:
try:
    import nltk
except:
    !pip install nltk
    import nltk

In [8]:
#This will load the csv
import csv
def loadDatasetCsv(filename):
    info = open(filename, "rb")
    has_header = csv.Sniffer().has_header(info.read(1024))
    info.seek(0)
    incsv = csv.reader(info)
    if has_header:
        next(incsv)  #Skip header
    dataset = list(incsv)
    return dataset
filename = 'train_set_x.csv'
dataset_without_classes = loadDatasetCsv(filename)
print('Loaded data file {0} with {1} rows').format(filename, len(dataset_without_classes))
print(dataset_without_classes[18])

Loaded data file train_set_x.csv with 276517 rows
['18', 'Was f\xc3\xbcr eine Mail Was soll da drin stehen']


In [9]:
#This will load the csv
import csv
def loadClassCsv(filename):
    info = open(filename, "rb")
    has_header = csv.Sniffer().has_header(info.read(1024))
    info.seek(0)
    incsv = csv.reader(info)
    if has_header:
        next(incsv)  #Skip header
    dataset = list(incsv)
    return dataset
filename = 'train_set_y.csv'
classds = loadClassCsv(filename)
print('Loaded data file {0} with {1} rows').format(filename, len(classds))
print(classds[18])

Loaded data file train_set_y.csv with 276517 rows
['18', '3']


In [10]:
#get Id from list object
def getId(line):
    info = csv.reader(line, delimiter=',')
    new_line = list(info)
    return new_line[0]

In [11]:
#get Utterance from list object
def getUtt(line):
    info = csv.reader(line, delimiter=',')
    new_line = list(info)
    return new_line[1]
#print(getUtt(dataset[1]))

In [12]:
def getClass(line):
    info = csv.reader(line, delimiter=',')
    new_line = list(info)
    new_line_item = ' '.join(new_line[1])
    return new_line_item

In [13]:
def appendClass(dataset):
    for i in range(len(dataset)):
        dataset[i].append(getClass(classds[i])) #Can assume this since they're all in the same position
    return dataset
dataset_with_classes = appendClass(dataset_without_classes)
print(dataset_with_classes[20])

['20', 'Pas mal pour une ligne Tu sugg\xc3\xa8res que Collard choisit', '1']


In [14]:
#Separate by class
#Assuming last index has the class nb
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (int(vector[-1]) not in separated):
            separated[int(vector[-1])] = []
        separated[int(vector[-1])].append(vector)
    return separated

separated = separateByClass(dataset_with_classes)
print(separated[0])

In [61]:
def createBigrams(ds, n):
    text = getUtt(ds[n])
    vector = ds[n]
    class1 = vector[-1]
    textNeeded = ''.join(text)
    textNeeded = textNeeded.replace(" ","")
    n = 2
    bigrams = [[(textNeeded[i:i+n]),class1] for i in range(0,len(textNeeded), 1)]
    return bigrams
all_ngrams = []
for i in range(len(dataset_with_classes)):
    all_ngrams.extend(createBigrams(dataset_with_classes,i))
print(all_ngrams_with_classes[34])
# for i in range(len(all_ngrams)):
#     if "é" in all_ngrams[i]:
#         print(all_ngrams[i])
#         del all_ngrams[i]

In [17]:
def calculatePrior(classDataset, language):
    print("Calculating prior")
    sum = len(classDataset)
    occurence = 0
    for i in range(sum):
        if(getClass(classDataset[i]) == language):
            occurence = occurence + 1
    prior = occurence/float(sum)
    return prior
# nb = calculatePrior(classds, '2')
# print(nb)

In [ ]:
def calculateLikelihood(all_seq, data, class_wanted):
    print("Calculating likelihood")
    separated = separateByClass(all_seq)
    sum = len(separated[class_wanted])
    arr = separated[class_wanted]
    occurence = 0
    for i in range(sum):
        if data in arr[i]:
            occurence += 1
    h = occurence/float(sum)
    return h
    
# nb = calculateLikelihood(all_ngrams, "er", 1)
# print(nb)

In [63]:
def classify_a_bigram(bigram):
    arr_of_possibilites = [0,0,0,0,0]
    arr_of_langs = ["Slovak","French","Spanish","German","Polish"]
    for i in range(len(arr_of_possibilites)):
        print("Calculation of stats for " + arr_of_langs[i])
        prior = calculatePrior(classds, str(i))
        likelihood = calculateLikelihood(all_ngrams, bigram, i)
        arr_of_possibilites[i] = prior*likelihood
    return numpy.argmax(arr_of_possibilites)
#print(classify_a_bigram("os"))

In [64]:
import csv
def loadDatasetCsv(filename):
    info = open(filename, "rb")
    has_header = csv.Sniffer().has_header(info.read(1024))
    info.seek(0)
    incsv = csv.reader(info)
    if has_header:
        next(incsv)  #Skip header
    dataset = list(incsv)
    return dataset
filename = 'test_set_x.csv'
dataset_test_set = loadDatasetCsv(filename)
print('Loaded data file {0} with {1} rows').format(filename, len(dataset_test_set))
length = len(dataset_test_set)

for i in range(length):
    ngrams = createBigrams(dataset_test_set,i)
    array_of_languages = []
    for i in range(len(ngrams)):
        array_of_languages.append(classify_a_bigram(ngrams[i]))
    print(array_of_languages)
    with open('output.csv', "wb") as csvfile:
        writer = csv.writer(csvfile)
        header = "Id,Category"
        writer.writerow([header])
        writer.writerow(getId(dataset_test_set[i]) + dataset_test_set[1])
    
# ngrams = createBigrams(dataset_test_set,3)
# print(ngrams)
# all_ngrams_with_classes = []
# for i in range(len(dataset_test_set)):
#     all_ngrams_with_classes.extend(createBigrams(dataset_test_set,i))

Loaded data file test_set_x.csv with 118508 rows


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/karinemellata/anaconda/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/Users/karinemellata/anaconda/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/Users/karinemellata/anaconda/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/Users/karinemellata/anaconda/lib/python2.7/inspect.py", line 1048, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/Users/karinemellata/anaconda/lib/python2.7/inspect.py", line 1008, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/Users/karinemellata/anaconda/lib/python2.7/inspect.py", line 453, in getsourcefile



KeyboardInterrupt

